# Matrix Approximation

In [1]:
from pyqubo import Array, Binary, Model
from matplotlib import pyplot as plt
import matplotlib.ticker as ticker
import requests
import json
import re
import time
import numpy as np
import scipy as sp

from lib.util import *

In [7]:
dimension = (3, 5)
precision = 21
starting_point = 0

X = generate_random_matrix(dimension)
approximated_X = generate_target_matrix(dimension, precision, starting_point)
hamiltonian = np.trace(np.inner(X-approximated_X, X-approximated_X))

In [8]:
start_time = time.time()

model = hamiltonian.compile()
qubo, offset = model.to_qubo()
matrix_term = get_matrix_term(qubo, model.variables)

end_time = time.time()
elapsed_time = end_time - start_time
print("Elapsed time: ", elapsed_time, " seconds")

Elapsed time:  0.03492307662963867  seconds


In [9]:
problem_body = {}
DA_Solver = {}
DA_Solver["time_limit_sec"]= 100
DA_Solver["penalty_coef"]=10000
DA_Solver["num_run"] = 16
DA_Solver["num_group"] = 16
DA_Solver['gs_level'] = 50
DA_Solver['gs_cutoff'] = 80000
problem_body["fujitsuDA3"]=DA_Solver
problem_body["binary_polynomial"] = {'terms' : matrix_term}

In [10]:
response = post_solve(problem_body)
job_id = response.json()['job_id']

{'message': 'success'}


In [11]:
solution = get_solution(job_id)
configuration = solution['qubo_solution']['solutions'][0]['configuration']

In [14]:
def decode(configuration, dimension, precision, start_point):
    binary_code = create_binary_representated_vector(precision, start_point)
    nrows, ncols = dimension
    
    solution_dict = {}
    for key, value in configuration.items():
        if value:
            solution_dict[model.variables[int(key)]] = 1
        else:
            solution_dict[model.variables[int(key)]] = 0
            
    # binary_representation_columns is an array whose element are represented as a matrix
    # the matrix multiply binary_code would become a column
    binary_representation_columns = []
    for i in range(ncols):
        binary_representation_columns.append(np.zeros((nrows, precision)))

    for key, value in solution_dict.items():
        # get the name of column
        indexes = re.findall(r'(\d+)', key)
        indexes = [int(i) for i in indexes]
        col = indexes[0]
        binary_representation_columns[col][indexes[1]][indexes[2]] = value

    columns = []
    for mat in binary_representation_columns:
        columns.append(np.array((mat * np.matrix(binary_code).transpose()).transpose())[0])

    return np.matrix(np.column_stack(columns)), binary_representation_columns, binary_code

In [16]:
mat, b_columns, b_code = decode(configuration, dimension, precision, starting_point)

In [19]:
np.trace(np.inner(mat-X, mat-X))

1.1945419345404711e-12